<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">


# Telecomm EDA Challenge Lab

_Author: Alex Combs (NYC) _

---

Let's do some Exploratory Data Analysis (EDA)! As a data scientist, you often may find yourself given a data set you've never seen before, and asked to do a rapid analysis. This is today's goal.

# Prompt

You work for a telecommunications company. The company has been storing metadata about customer phone usage, as part of the regular course of business. Currently, this data is sitting in an unsecured database. The company doesn't want to pay to increase their database security, because they don't think there's really anything to be learned from the metadata.

They are under pressure from "right to privacy" organizations to beef up the database security. These organizations argue that you can learn a lot about a person from their cell phone metadata.

The telecom company wants to understand if this is true, and they want your help. They will give you one person's metadata for 2014 and want to see what you can learn from it.

Working in teams, create a report revealing everything you can about the person. Prepare a presentation, with slides, showcasing your findings.


# The Data

The [person's metadata](./datasets/metadata.csv) has the following fields:

| Field Name          | Description
| ---                 | ---
| **Cell Cgi**        | cell phone tower identifier
| **Cell Tower**      | cell phone tower location
| **Comm Identifier** |	de-identified recipient of communication
| **Comm Timedate String** | time of communication
| **Comm Type	Id**  | type of communication
| **Latitude**        | latitude of communication
| **Longitude**       | longitude of communication


# Hints

This is totally open-ended! If you're totally stumped -- and only if stumped -- should you look below for prompts. As a starting point, given that you have geo-locations, consider investigating ways to display this type of information (i.e. mapping functionality).

<font color='white'>
Well for starters, he's in Australia!

Ideas for things to look into:
- where does he work?
- where does he live?
- who does he contact most often?
- what hours does he work?
- did he move?
- did he go on holiday?  If so, where did he go?
- did he get a new phone?

Challenges:
- how does he get to work?
- where does his family live?
- if he went on holiday, can you find which flights he took?
- can you guess who some of his contacts are, based on the frequency, location, time and mode (phone/text) of communications?


If you're stuck on how to map the data, you can try "basemap" or "gmplot", or anything else you find online.
</font>

In [1]:
# Telecomm EDA Challenge Lab

# EXECUTIVE SUMMARY 
# The data contains personal information that needs to be proected and secured. 
# Investment must be made quickly to secure this data.

# SUMMARY OVERVIEW OF THE DATA
# Records from the individual account taken at 6,382 unique time stamps over the course of the year
# Records taken from at 70 different locations in over this time 

# EXAMPLES OF LOCATIONS 

# MELBORNE INTERNATIONAL AIRPORT
# Only one record from here
# Call made from the location at 2014-08-30 10:32:00  

# TASMANIA
# 149B Macquarie Street, Hobart, Tasmania, 7000

# 4 HAYES ST, SYDNEY
# Location had the most records
# 4118 records from 4 Hayes Street, Sydney, New South Wales, 2093
# Records from between 2014-09-24 and 2015-03-25 only 
# Was only at this location location for a 6 month period 
# 2740 different time stamps from early morning to late at night
# Only used the phone here twice

# REGENT STREET, SYDNEY 
# 998 records from this location 
# Records across the whole year 
# 606 different time stamps from early morning 
# Used the phone 15 times

# 103-109 GEORGE ST, SYDNEY 
# 334 reords over the course of the year from from 103-109 George Street, Sydney
# records include phone, internet and SMS from this location 

# MORE SUMMARY DATA
# 10476 records over the course of a year
# Full data except for the identifier column which has 9102 nulls
# 131 Cell Cgi IDs across 71 towers 
# 130 identifiers 
# 87% internet of records track internet, 7% phone and 6% SMS 

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import os

plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Billboard data CSV:
metadata_csv = './metadata.csv'

# We need to use encoding='latin-1' to deal with non-ASCII characters.
df = pd.read_csv(metadata_csv, encoding='latin-1')

In [3]:
df.head(1)

,Cell Cgi,Cell Tower Location,Comm Identifier,Comm Timedate String,Comm Type,Latitude,Longitude
0,50501015388B9,REDFERN TE,f1a6836c0b7a3415a19a90fdd6f0ae18484d6d1e,4/1/14 9:40,Phone,-33.892933,151.202296


In [4]:
df.rename(columns={'Cell Cgi':'cgi','Cell Tower Location':'tower', 'Comm Identifier':'identifier', 'Comm Timedate String':'time','Comm Type':'type', 'Latitude':'lat', 'Longitude':'lon'}, inplace=True)
df.head(1)

,cgi,tower,identifier,time,type,lat,lon
0,50501015388B9,REDFERN TE,f1a6836c0b7a3415a19a90fdd6f0ae18484d6d1e,4/1/14 9:40,Phone,-33.892933,151.202296


In [5]:
df['time_stamp'] = pd.to_datetime(df['time'])
df.head(1)

,cgi,tower,identifier,time,type,lat,lon,time_stamp
0,50501015388B9,REDFERN TE,f1a6836c0b7a3415a19a90fdd6f0ae18484d6d1e,4/1/14 9:40,Phone,-33.892933,151.202296,2014-04-01 09:40:00


In [6]:
df.shape

(10476, 8)

In [7]:
df.dtypes.head

<bound method NDFrame.head of cgi                   object
tower                 object
identifier            object
time                  object
type                  object
lat                  float64
lon                  float64
time_stamp    datetime64[ns]
dtype: object>

In [8]:
print((df.isnull().sum().head(15)))

cgi              0
tower            0
identifier    9102
time             0
type             0
lat              0
lon              0
time_stamp       0
dtype: int64


In [9]:
df['cgi'].value_counts()

505012056EF02    4118
5.05E+12         1627
5050101532B23     998
505012950037C     711
5050129505A15     491
                 ... 
5050120576A03       1
505010152046C       1
5050101530DB8       1
5050101536A43       1
50501202B6D03       1
Name: cgi, Length: 131, dtype: int64

In [10]:
df['tower'].value_counts()

BALGOWLAH HAYES ST           4301
CHIPPENDALE                  1084
SUNDERLAND ST                 723
REDFERN TE                    712
HAYMARKET #                   563
                             ... 
NEWTOWN TE                      1
MASCOT 201 COWARD ST            1
MELB AIRPORT TERM               1
REDFERN; 495 CLEVELAND ST       1
CARLTON COLLEGE SQ              1
Name: tower, Length: 71, dtype: int64

In [11]:
df['identifier'].value_counts()

bc0b01860486b0f0a240ce8419d3d7553fe404ab    219
12e3d1b0c95aa32b6890c4455918dfc10e09fb51    146
91aba4a11359ff3af7902428d20cfa7e676c36e7    144
a24a4646d074a779b45b34b943a47bf33168f791    133
6bbc17070aa91e2dab7909b96c6eecbd6109ba56     83
                                           ... 
caca077c49aa8bc43ed6a93fb15e37027d116161      1
55c9fbf7495f31d631bf0f26a00e4494f1c5715e      1
4b12e682d816d603ca3820581e31fe1ce71d3e7f      1
3274802c767c077c0ce3f6c2f76fabe35129f696      1
506f74c16a228ac966a2c785cc5cdb32946c769d      1
Name: identifier, Length: 130, dtype: int64

In [12]:
df['type'].value_counts()

Internet    9102
Phone        717
SMS          657
Name: type, dtype: int64

In [13]:
df['type'].value_counts() /df['type'].value_counts().sum()

Internet    0.868843
Phone       0.068442
SMS         0.062715
Name: type, dtype: float64

In [14]:
df['time'].value_counts()

3/31/15 18:59     26
12/11/14 16:08    11
1/24/15 20:35     11
2/3/15 8:44       10
1/30/15 20:39      9
                  ..
1/31/15 10:25      1
12/23/14 10:58     1
3/26/15 13:35      1
9/5/14 16:41       1
10/24/14 13:27     1
Name: time, Length: 6382, dtype: int64

In [15]:
df['time_stamp'].describe()

count                   10476
unique                   6382
top       2015-03-31 18:59:00
freq                       26
first     2014-04-01 09:40:00
last      2015-04-01 23:49:00
Name: time_stamp, dtype: object

In [16]:
df['lat'].value_counts()

-33.788150    4301
-33.884171    1084
-42.843380     723
-33.892933     712
-33.880329     563
              ... 
-37.777800       1
-37.670418       1
-33.892330       1
-33.557310       1
-37.792900       1
Name: lat, Length: 70, dtype: int64

In [17]:
df['lon'].value_counts()

151.266540    4301
151.202350    1084
147.295690     723
151.202296     712
151.205690     563
              ... 
151.210100       1
151.188043       1
144.848243       1
151.216530       1
151.238020       1
Name: lon, Length: 70, dtype: int64

In [18]:
df['lon'].describe()

count    10476.000000
mean       150.612841
std          1.470169
min        144.848243
25%        151.202296
50%        151.266540
75%        151.266540
max        151.289340
Name: lon, dtype: float64

In [19]:
# Lon Describe 
df.loc[df['lon'] == 144.848243]


,cgi,tower,identifier,time,type,lat,lon,time_stamp
577,5050122513CAB,MELB AIRPORT TERM,6bbc17070aa91e2dab7909b96c6eecbd6109ba56,8/30/14 10:32,Phone,-37.670418,144.848243,2014-08-30 10:32:00


In [20]:
df['lat'].describe()

count    10476.000000
mean       -35.136188
std          3.141723
min        -42.884810
25%        -33.884603
50%        -33.796610
75%        -33.788150
max        -33.557310
Name: lat, dtype: float64

In [21]:
# Lat Describe  
df.loc[df['lat'] == -33.884603]

,cgi,tower,identifier,time,type,lat,lon,time_stamp


In [22]:
# concatenate to find max and min 
df['lat_lon'] = df['lat'].astype(str) + df['lon'].astype(str)
df.head(1)
df.head(1)

,cgi,tower,identifier,time,type,lat,lon,time_stamp,lat_lon
0,50501015388B9,REDFERN TE,f1a6836c0b7a3415a19a90fdd6f0ae18484d6d1e,4/1/14 9:40,Phone,-33.892933,151.202296,2014-04-01 09:40:00,-33.89293336151.20229619999998


In [23]:
df['lat_lon'].max()

'-42.884809999999995147.32748'

In [24]:
df['lon_lat'] = df['lon'].astype(str) + df['lat'].astype(str)
df.head(1)

,cgi,tower,identifier,time,type,lat,lon,time_stamp,lat_lon,lon_lat
0,50501015388B9,REDFERN TE,f1a6836c0b7a3415a19a90fdd6f0ae18484d6d1e,4/1/14 9:40,Phone,-33.892933,151.202296,2014-04-01 09:40:00,-33.89293336151.20229619999998,151.20229619999998-33.89293336


In [25]:
df['lat_lon'].describe()

count                  10476
unique                    70
top       -33.78815151.26654
freq                    4301
Name: lat_lon, dtype: object

In [26]:
# Looking up a location in REDFERN 
df.loc[df['cgi'] == '50501015388B9']

,cgi,tower,identifier,time,type,lat,lon,time_stamp,lat_lon,lon_lat
0,50501015388B9,REDFERN TE,f1a6836c0b7a3415a19a90fdd6f0ae18484d6d1e,4/1/14 9:40,Phone,-33.892933,151.202296,2014-04-01 09:40:00,-33.89293336151.20229619999998,151.20229619999998-33.89293336
1,50501015388B9,REDFERN TE,62157ccf2910019ffd915b11fa037243b75c1624,4/1/14 9:42,Phone,-33.892933,151.202296,2014-04-01 09:42:00,-33.89293336151.20229619999998,151.20229619999998-33.89293336
52,50501015388B9,REDFERN TE,6bbc17070aa91e2dab7909b96c6eecbd6109ba56,4/13/14 10:58,Phone,-33.892933,151.202296,2014-04-13 10:58:00,-33.89293336151.20229619999998,151.20229619999998-33.89293336
71,50501015388B9,REDFERN TE,bc0b01860486b0f0a240ce8419d3d7553fe404ab,4/14/14 21:11,SMS,-33.892933,151.202296,2014-04-14 21:11:00,-33.89293336151.20229619999998,151.20229619999998-33.89293336
140,50501015388B9,REDFERN TE,cd3b39466869088df4904451c626591cc500e4ba,4/26/14 11:57,SMS,-33.892933,151.202296,2014-04-26 11:57:00,-33.89293336151.20229619999998,151.20229619999998-33.89293336
...,...,...,...,...,...,...,...,...,...,...
10471,50501015388B9,REDFERN TE,NaN,4/1/15 22:40,Internet,-33.892933,151.202296,2015-04-01 22:40:00,-33.89293336151.20229619999998,151.20229619999998-33.89293336
10472,50501015388B9,REDFERN TE,NaN,4/1/15 22:41,Internet,-33.892933,151.202296,2015-04-01 22:41:00,-33.89293336151.20229619999998,151.20229619999998-33.89293336
10473,50501015388B9,REDFERN TE,NaN,4/1/15 22:44,Internet,-33.892933,151.202296,2015-04-01 22:44:00,-33.89293336151.20229619999998,151.20229619999998-33.89293336
10474,50501015388B9,REDFERN TE,NaN,4/1/15 22:45,Internet,-33.892933,151.202296,2015-04-01 22:45:00,-33.89293336151.20229619999998,151.20229619999998-33.89293336


In [27]:
df.loc[df['cgi'] == '50501015388B9']

,cgi,tower,identifier,time,type,lat,lon,time_stamp,lat_lon,lon_lat
0,50501015388B9,REDFERN TE,f1a6836c0b7a3415a19a90fdd6f0ae18484d6d1e,4/1/14 9:40,Phone,-33.892933,151.202296,2014-04-01 09:40:00,-33.89293336151.20229619999998,151.20229619999998-33.89293336
1,50501015388B9,REDFERN TE,62157ccf2910019ffd915b11fa037243b75c1624,4/1/14 9:42,Phone,-33.892933,151.202296,2014-04-01 09:42:00,-33.89293336151.20229619999998,151.20229619999998-33.89293336
52,50501015388B9,REDFERN TE,6bbc17070aa91e2dab7909b96c6eecbd6109ba56,4/13/14 10:58,Phone,-33.892933,151.202296,2014-04-13 10:58:00,-33.89293336151.20229619999998,151.20229619999998-33.89293336
71,50501015388B9,REDFERN TE,bc0b01860486b0f0a240ce8419d3d7553fe404ab,4/14/14 21:11,SMS,-33.892933,151.202296,2014-04-14 21:11:00,-33.89293336151.20229619999998,151.20229619999998-33.89293336
140,50501015388B9,REDFERN TE,cd3b39466869088df4904451c626591cc500e4ba,4/26/14 11:57,SMS,-33.892933,151.202296,2014-04-26 11:57:00,-33.89293336151.20229619999998,151.20229619999998-33.89293336
...,...,...,...,...,...,...,...,...,...,...
10471,50501015388B9,REDFERN TE,NaN,4/1/15 22:40,Internet,-33.892933,151.202296,2015-04-01 22:40:00,-33.89293336151.20229619999998,151.20229619999998-33.89293336
10472,50501015388B9,REDFERN TE,NaN,4/1/15 22:41,Internet,-33.892933,151.202296,2015-04-01 22:41:00,-33.89293336151.20229619999998,151.20229619999998-33.89293336
10473,50501015388B9,REDFERN TE,NaN,4/1/15 22:44,Internet,-33.892933,151.202296,2015-04-01 22:44:00,-33.89293336151.20229619999998,151.20229619999998-33.89293336
10474,50501015388B9,REDFERN TE,NaN,4/1/15 22:45,Internet,-33.892933,151.202296,2015-04-01 22:45:00,-33.89293336151.20229619999998,151.20229619999998-33.89293336


In [28]:
# HAYES LOCATION 
df_4_hayes_st = df.loc[df['cgi'] == '505012056EF02']

In [29]:
df_4_hayes_st['time_stamp'].describe()

count                    4118
unique                   2740
top       2015-02-03 08:44:00
freq                       10
first     2014-09-24 17:17:00
last      2015-03-25 16:36:00
Name: time_stamp, dtype: object

In [30]:
df_4_hayes_st['time_stamp'].value_counts()

2015-02-03 08:44:00    10
2014-12-04 06:52:00     9
2015-01-24 20:53:00     8
2014-11-09 13:49:00     7
2014-11-09 13:28:00     7
                       ..
2015-01-02 13:09:00     1
2015-02-08 09:40:00     1
2014-10-31 20:05:00     1
2014-11-16 07:29:00     1
2015-01-14 16:11:00     1
Name: time_stamp, Length: 2740, dtype: int64

In [31]:
df_4_hayes_st['type'].value_counts()

Internet    3929
SMS          187
Phone          2
Name: type, dtype: int64

In [32]:
# Regent Street, Sydney Chippendale, New South Wales, 2008
# Chippendale Sydney Australia
df_regent_st = df.loc[df['cgi'] == '5050101532B23']

In [33]:
df_regent_st['time_stamp'].describe()

count                     998
unique                    606
top       2015-01-30 16:01:00
freq                        6
first     2014-04-01 17:36:00
last      2015-04-01 17:29:00
Name: time_stamp, dtype: object

In [34]:
df_regent_st['time_stamp'].value_counts()

2015-01-30 16:01:00    6
2014-12-15 07:01:00    5
2014-11-11 15:14:00    5
2015-01-28 12:50:00    5
2014-11-17 15:20:00    5
                      ..
2014-11-27 12:11:00    1
2015-03-18 09:03:00    1
2014-12-17 06:15:00    1
2015-01-07 10:30:00    1
2015-03-19 06:59:00    1
Name: time_stamp, Length: 606, dtype: int64

In [35]:
df_regent_st['type'].value_counts()

Internet    981
Phone        15
SMS           2
Name: type, dtype: int64

In [210]:
# MAPPING
import gmplot
# also: pip install gmplot
# also tried: conda install -c mlgill gmplot
# conda install -c mlgill gmplot
# I am not sure which one worked!!!!

In [38]:
pip install gmplot

Note: you may need to restart the kernel to use updated packages.


In [225]:
conda install -c mlgill gmplot

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
Comparing specs that have this dependency:   0%|          | 0/2 [00:00<?, ?it/s]

Finding conflict paths:   0%|                             | 0/1 [00:00<?, ?it/s]

Finding shortest conflict path for wheel:   0%|           | 0/1 [00:00<?, ?it/s]| 

                                                                                

Finding conflict paths:   0%|                             | 0/1 [00:00<?, ?it/s]

Finding shortest conflict path for wheel:   0%|           | 0/1 [00:00<?, ?it/s]

                                                                                
Comparing specs

In [46]:
import gmplot

In [53]:
# AN EXAMPLE I FOUND TO BUILD OFF 
# NEW DATA FRAME FOR THE FILE BELOW 
data = pd.read_csv('./llatest.csv', sep=',')

In [50]:
# THIS IS TAKEN FROM THIS URL AND IT WORKS!!!!
# https://medium.com/@stevenvandorpe/gmplot-in-jupyter-installation-guide-and-package-exploration-338756e8f26

min_lat, max_lat, min_lon, max_lon = \
min(data['lat']), max(data['lat']), \
min(data['lon']), max(data['lon'])
## Create empty map with zoom level 16
mymap = gmplot.GoogleMapPlotter(
    min_lat + (max_lat - min_lat) / 2, 
    min_lon + (max_lon - min_lon) / 2, 
    16)

In [54]:
mymap.plot(data['lat'], data['lon'], 'blue', edge_width=1)
mymap.draw('my_gm_plot2.html')
os.system('my_gm_plot2.html')
# THIS CREATES A FILE IN THE FOLDER

32512

In [57]:
# TEST & APPLY TO TELECOMM DF
min_lat, max_lat, min_lon, max_lon = \
min(df['lat']), max(df['lat']), \
min(df['lon']), max(df['lon'])
## Create empty map with zoom level 16
mymap2 = gmplot.GoogleMapPlotter(
    min_lat + (max_lat - min_lat) / 2, 
    min_lon + (max_lon - min_lon) / 2, 
    16)
mymap2.plot(df['lat'], df['lon'], 'blue', edge_width=1)
mymap2.draw('telecoms_map.html')
os.system('telecoms_map.html')

32512